In [29]:
import pandas as pd
import numpy as np
from numpy.core.records import fromarrays
from scipy.io import savemat

In [30]:
choice = 0  #start from 0
devices = ['nfet_01v8_lvt', 'nfet_01v8', 'pfet_01v8_lvt', 'pfet_01v8']

# widths used for characterization and fringe cap parameters (fringe caps are not included in ngspice output)
w = np.array([5, 5, 5, 5])
nfing = np.array([1, 1, 1, 1])
cgdo_w = np.array([2.39289e-10, 2.44907e-10, 2e-11, 5.24893e-11])*1e-6

In [31]:
df_raw = pd.read_csv('./simulation/techsweep_'+devices[choice]+'.txt', sep='\s+')
par_names = df_raw.columns.to_list()
par_prefix = par_names[1].split('[')[0]
print(par_prefix)

@m.xm1.msky130_fd_pr__nfet_01v8_lvt


In [32]:
# remove extra headers in file body and unwanted columns
df_raw = df_raw[~df_raw['v-sweep'].str.contains('v-sweep')]
df = df_raw.drop(['v-sweep', 'v-sweep.1'], axis=1)
df = df.apply(pd.to_numeric).abs()

# rename columns for readability
df.columns = df.columns.str.removeprefix(par_prefix+'[')
df.columns = df.columns.str.removesuffix(']')
df

,capbd,capbs,cgg,gds,gm,gmbs,id,l,vth,b,d,g
0,2.820000e-15,2.820000e-15,1.619000e-15,6.077000e-11,0.000000,0.000000,0.000000,1.500000e-07,0.6197,0.0,0.0,0.000
1,2.820000e-15,2.820000e-15,1.621000e-15,1.156000e-10,0.000000,0.000000,0.000000,1.500000e-07,0.6197,0.0,0.0,0.025
2,2.820000e-15,2.820000e-15,1.628000e-15,2.200000e-10,0.000000,0.000000,0.000000,1.500000e-07,0.6197,0.0,0.0,0.050
3,2.820000e-15,2.820000e-15,1.641000e-15,4.186000e-10,0.000000,0.000000,0.000000,1.500000e-07,0.6197,0.0,0.0,0.075
4,2.820000e-15,2.820000e-15,1.659000e-15,7.963000e-10,0.000000,0.000000,0.000000,1.500000e-07,0.6197,0.0,0.0,0.100
...,...,...,...,...,...,...,...,...,...,...,...,...
255834,1.471000e-15,2.284000e-15,8.809000e-14,4.189000e-06,0.000412,0.000070,0.000276,3.000000e-06,0.5237,0.4,1.8,1.700
255835,1.471000e-15,2.284000e-15,8.817000e-14,4.362000e-06,0.000417,0.000072,0.000287,3.000000e-06,0.5237,0.4,1.8,1.725
255836,1.471000e-15,2.284000e-15,8.824000e-14,4.552000e-06,0.000422,0.000074,0.000297,3.000000e-06,0.5237,0.4,1.8,1.750
255837,1.471000e-15,2.284000e-15,8.832000e-14,4.760000e-06,0.000426,0.000076,0.000308,3.000000e-06,0.5237,0.4,1.8,1.775


In [33]:
# sweep variable vectors
l = (np.unique(df['l'])/1e-6).round(3)
vgs = np.unique(df['g'])
vds = np.unique(df['d'])
vsb = np.unique(df['b'])

In [34]:
# data
# ngspice sweep order is vgs, vds, vsb, l
id = np.reshape(df['id'].values, (len(vgs), len(vds), len(vsb), len(l)), order='F')
vt = np.reshape(df['vth'].values, (len(vgs), len(vds), len(vsb), len(l)), order='F')
gm = np.reshape(df['gm'].values, (len(vgs), len(vds), len(vsb), len(l)), order='F')
gmb = np.reshape(df['gmbs'].values, (len(vgs), len(vds), len(vsb), len(l)), order='F')
gds = np.reshape(df['gds'].values, (len(vgs), len(vds), len(vsb), len(l)), order='F')
cgg = np.reshape(df['cgg'].values, (len(vgs), len(vds), len(vsb), len(l)), order='F') + 2*w[choice]*cgdo_w[choice]
cdd = np.reshape(df['capbd'].values, (len(vgs), len(vds), len(vsb), len(l)), order='F') + w[choice]*cgdo_w[choice]
css = np.reshape(df['capbs'].values, (len(vgs), len(vds), len(vsb), len(l)), order='F') + w[choice]*cgdo_w[choice]
cgd = np.full((len(vgs), len(vds), len(vsb), len(l)), w[choice]*cgdo_w[choice])

# output order for mat file is l, vgs, vds, vsb
id = np.transpose(id, (3, 0, 1, 2))
vt = np.transpose(vt, (3, 0, 1, 2))
gm = np.transpose(gm, (3, 0, 1, 2))
gmb = np.transpose(gmb, (3, 0, 1, 2))
gds = np.transpose(gds, (3, 0, 1, 2))
cgg = np.transpose(cgg, (3, 0, 1, 2))
cdd = np.transpose(cdd, (3, 0, 1, 2))
css = np.transpose(css, (3, 0, 1, 2))
cgd = np.transpose(cgd, (3, 0, 1, 2))

In [35]:
dic = {
  "INFO": "SkyWater, 130nm CMOS, BSIM4",
  "CORNER": "NOM",
  "TEMP": 300.0,
  "VGS": vgs,
  "VDS": vds,
  "VSB": vsb,
  "L": l,
  "W": w[choice],
  "NFING": nfing[choice],
  "ID": id,
  "VT": vt,
  "GM": gm,
  "GMB": gmb,
  "GDS": gds,
  "CGG": cgg,
  "CDD": cdd,
  "CSS": css,
  "CGD": cgd
}

savemat('./simulation/'+devices[choice]+'.mat', {devices[choice]: dic})